<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0
    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-16 13:44:25
-------------------
qualified stocks: 96
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  70.78 K
Current:  1.28 C
-------------------
Today PnL: 1.30 L (1.02%)
Current PnL: -15.75 L (-11.17%)
CY Booked + Current PnL: -12.88 L (-9.13%)
-------------------
Total profit:  4.51 L
Total loss:  -20.25 L
-------------------
Total Booked + Current PnL: 14.10 L (12.37%)
Total Booked PnL: 29.85 L (26.17%)
Curr Year Booked PnL: 2.87 L (2.24%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.07 C
Est FTT PnL: 78.84 L (61.65%)
-------------------
Est LTT:  2.68 C
Est LTT PnL: 1.40 C (109.72%)
Deployed:  1.14 C
Current:  1.28 C
CAGR/XIRR %: 8.28%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,MRF,114777.60,150504.12,14.94,X-MC,79.17,141210.0,26432.0,9292.0,0.05,23.03,6.58,31.13,220.0,2.84,1.11,37.55,AR,ATH,TYRES
63,RELIANCE,1288.53,1526.00,6.87,X-LC,92.71,230127.0,26539.0,10977.0,0.16,13.04,4.77,18.43,157.0,2.42,1.81,25.36,XY25,NTT,REFINERIES
70,SBILIFE,1496.49,1924.99,0.88,H-LC,77.08,149906.0,24201.0,11798.0,1.67,19.25,7.87,28.63,147.0,2.05,1.18,33.55,AR,ATH,INSURANCE
57,PIDILITIND,2504.06,3576.00,-16.34,X-LC,32.29,92760.0,17638.0,14517.0,-0.63,23.48,15.65,42.81,120.0,1.21,0.73,16.33,X40,BTT,CHEMICALS
6,ANGELONE,2328.67,3033.00,10.77,H-SC,94.79,180304.0,28940.0,16840.0,7.99,19.12,9.34,30.25,111.0,1.72,1.42,42.71,X40N,NTT,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,TITAGARH,1097.23,1548.00,-9.03,H-SC,87.50,159740.0,-32275.0,111163.0,12.82,-16.81,69.59,41.08,95.0,-0.29,1.26,35.20,XY24,NTT,ENGINEERING
90,VALIANTORG,512.64,838.00,-327.70,H-SC,55.21,96720.0,-36566.0,121161.0,8.22,-27.43,125.27,63.47,161.0,-0.30,0.76,62.13,XR,NTT,CHEMICALS
6,ANGELONE,2328.67,3033.00,10.77,H-SC,94.79,180304.0,28940.0,16840.0,7.99,19.12,9.34,30.25,111.0,1.72,1.42,42.71,X40N,NTT,FINANCE
77,STARHEALTH,551.56,761.00,-0.95,M-SC,85.42,161898.0,-38318.0,114349.0,6.17,-19.14,70.63,37.97,62.0,-0.34,1.27,29.99,XY24,NTT,INSURANCE
18,CAMS,3643.00,5250.99,-2.89,H-SC,20.83,109138.0,7134.0,37893.0,4.60,6.99,34.72,44.14,22.0,0.19,0.86,24.37,XR,ATH,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1022.60,1486.00,3.34,X-MC,2.08,182142.0,26707.0,43732.0,-3.62,17.18,24.01,45.32,201.0,0.61,1.43,23.93,XY24,NTT,HEALTHCARE
40,INDUSINDBK,1354.50,1800.00,-41.30,H-MC,7.29,43602.0,-33604.0,58998.0,-1.99,-43.53,135.31,32.89,24.0,-0.57,0.34,20.08,XR,NTT,BANKS
79,SYMPHONY,1311.84,2037.54,-21.02,M-SC,72.92,147775.0,-3087.0,86537.0,-1.82,-2.05,58.56,55.32,15.0,-0.04,1.16,30.03,X40N,ATH,DURABLES
68,SATIN,199.77,274.00,-15.90,H-SC,64.58,167211.0,-28763.0,101581.0,-1.61,-14.68,60.75,37.16,103.0,-0.28,1.31,24.87,XY24,NTT,FINANCE
41,INFY,1432.81,2275.00,-11.34,X-LC,73.96,220774.0,21613.0,95441.0,-1.53,10.85,43.23,58.78,10.0,0.23,1.74,15.26,X40,BTT,IT


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,TATAMOTORS,776.02,1065.00,-51.41,M-LC,95.83,194126.0,-11519.0,88094.0,0.61,-5.60,45.38,37.24,1.0,-0.13,1.53,26.36,XY24,NTT,AUTO
38,INDIAMART,2311.97,4911.36,-52.21,H-SC,66.67,103180.0,1453.0,112920.0,0.62,1.43,109.44,112.43,2.0,0.01,0.81,21.78,AR,ATH,MISC
2,ABB,5319.37,8953.12,-36.47,H-LC,62.50,145450.0,12466.0,78383.0,1.45,9.37,53.89,68.31,4.0,0.16,1.14,18.60,AR,ATH,ELECTRICAL
83,TCS,3928.29,4998.00,-14.87,X-LC,61.46,192029.0,-20099.0,77868.0,-0.67,-9.47,40.55,27.23,5.0,-0.26,1.51,10.04,X40,BTT,IT
49,LICI,777.74,983.00,-13.28,H-LC,54.17,191744.0,17530.0,28455.0,1.87,10.06,14.84,26.39,6.0,0.62,1.51,17.39,XY25,NTT,INSURANCE


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,MRF,114777.60,150504.12,14.94,X-MC,79.17,141210.0,26432.0,9292.0,0.05,23.03,6.58,31.13,220.0,2.84,1.11,37.55,AR,ATH,TYRES
57,PIDILITIND,2504.06,3576.00,-16.34,X-LC,32.29,92760.0,17638.0,14517.0,-0.63,23.48,15.65,42.81,120.0,1.21,0.73,16.33,X40,BTT,CHEMICALS
58,POLYCAB,5005.92,7541.35,9.13,H-MC,83.33,170800.0,30634.0,40360.0,0.96,21.86,23.63,50.65,55.0,0.76,1.34,30.35,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,-0.04,M-LC,81.25,193307.0,42101.0,23564.0,0.34,27.84,12.19,43.43,37.0,1.79,1.52,33.99,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,37.37,M-SC,40.62,142253.0,4675.0,89733.0,2.92,3.40,63.08,68.62,79.0,0.05,1.12,23.32,X40N,NTT,TEXTILES
6,ANGELONE,2328.67,3033.00,10.77,H-SC,94.79,180304.0,28940.0,16840.0,7.99,19.12,9.34,30.25,111.0,1.72,1.42,42.71,X40N,NTT,FINANCE
58,POLYCAB,5005.92,7541.35,9.13,H-MC,83.33,170800.0,30634.0,40360.0,0.96,21.86,23.63,50.65,55.0,0.76,1.34,30.35,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,-0.04,M-LC,81.25,193307.0,42101.0,23564.0,0.34,27.84,12.19,43.43,37.0,1.79,1.52,33.99,XY24,NTT,MISC
28,GILLETTE,8109.44,10330.69,-0.40,H-SC,70.83,146948.0,9088.0,28670.0,4.57,6.59,19.51,27.39,68.0,0.32,1.16,28.63,X40,ATH,FMCG


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,3643.00,5250.99,-2.89,H-SC,20.83,109138.0,7134.0,37893.0,4.60,6.99,34.72,44.14,22.0,0.19,0.86,24.37,XR,ATH,MISC
95,WIPRO,243.46,420.00,-7.89,M-LC,34.38,157592.0,6647.0,102813.0,-0.93,4.40,65.24,72.51,52.0,0.06,1.24,18.65,XR,NTT,IT
12,BANKINDIA,113.49,190.00,-33.09,M-MC,17.71,130347.0,1195.0,85873.0,0.43,0.93,65.88,67.42,41.0,0.01,1.02,26.77,XR,NTT,BANKS
38,INDIAMART,2311.97,4911.36,-52.21,H-SC,66.67,103180.0,1453.0,112920.0,0.62,1.43,109.44,112.43,2.0,0.01,0.81,21.78,AR,ATH,MISC
33,HEROMOTOCO,4311.81,6039.03,-0.95,H-MC,80.21,151375.0,462.0,59990.0,0.00,0.31,39.63,40.06,14.0,0.01,1.19,23.82,AR,ATH,AUTO


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
90,VALIANTORG,512.64,838.00,-327.70,H-SC,55.21,96720.0,-36566.0,121161.0,8.22,-27.43,125.27,63.47,161.0,-0.30,0.76,62.13,XR,NTT,CHEMICALS
87,UJJIVANSFB,52.77,60.00,40.02,M-SC,82.29,121527.0,-20952.0,40468.0,-0.88,-14.71,33.30,13.70,214.0,-0.52,0.96,43.44,X40N,NTT,BANKS
12,BANKINDIA,113.49,190.00,-33.09,M-MC,17.71,130347.0,1195.0,85873.0,0.43,0.93,65.88,67.42,41.0,0.01,1.02,26.77,XR,NTT,BANKS
54,MRF,114777.60,150504.12,14.94,X-MC,79.17,141210.0,26432.0,9292.0,0.05,23.03,6.58,31.13,220.0,2.84,1.11,37.55,AR,ATH,TYRES
70,SBILIFE,1496.49,1924.99,0.88,H-LC,77.08,149906.0,24201.0,11798.0,1.67,19.25,7.87,28.63,147.0,2.05,1.18,33.55,AR,ATH,INSURANCE


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,718.00,-876.84,H-SC,100.00,74002.0,-21314.0,66010.0,3.74,-22.36,89.20,46.89,82.0,-0.32,0.58,45.96,X40N,NTT,MISC
81,TATAELXSI,7332.28,9000.00,-17.32,X-MC,98.96,86814.0,-15838.0,39188.0,0.33,-15.43,45.14,22.74,47.0,-0.40,0.68,30.49,X40N,NTT,IT
80,TANLA,917.30,1963.11,-39.00,H-SC,97.92,149775.0,-89640.0,362590.0,3.13,-37.44,242.09,114.01,20.0,-0.25,1.18,37.37,AR,ATH,IT
76,SONACOMS,578.05,1006.00,-27.10,M-MC,96.88,94273.0,-6886.0,81782.0,0.93,-6.81,86.75,74.03,49.0,-0.08,0.74,32.49,AR,BTT,AUTO
82,TATAMOTORS,776.02,1065.00,-51.41,M-LC,95.83,194126.0,-11519.0,88094.0,0.61,-5.60,45.38,37.24,1.0,-0.13,1.53,26.36,XY24,NTT,AUTO


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,17.13
1,25,38.55
2,50,67.63


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    34.63
X40N    15.25
AR      15.07
X40     14.25
XR      10.24
XY25     8.26
X200     1.19
SR       1.11
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    22.48
M-SC    17.70
X-LC    16.07
H-MC     8.91
M-LC     8.28
X-MC     6.65
H-LC     5.89
M-MC     5.51
X-SC     2.79
L-SC     2.26
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,11.62,-0.47,32.43
BANKS,10.86,-9.05,62.53
IT,9.14,-19.15,85.62
FINANCE,7.44,-17.01,69.81
MISC,7.13,-24.74,78.41
INSURANCE,6.13,1.08,28.75
PAINTS,5.55,-16.55,51.04
AUTO,4.37,-8.85,56.29
ELECTRICAL,4.09,-6.74,61.13


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2948083.0,27
AR,1295808.0,16
XR,1239732.0,15
X40N,1227603.0,18
X40,782588.0,12
SR,204198.0,2
XY25,119245.0,5
X200,66928.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2331713.0,22
M-SC,1988172.0,21
X-LC,697767.0,12
M-MC,566440.0,5
H-MC,473061.0,8
X-SC,353887.0,4
M-LC,327229.0,6
L-SC,309842.0,4
X-MC,309360.0,6


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      897117.0      6
M-SC       XY24      743310.0      6
H-SC       AR        535678.0      3
           X40N      522591.0      7
M-SC       X40N      449987.0      6
           XR        423886.0      5
M-MC       XY24      398785.0      3
X-LC       X40       375350.0      5
H-SC       XR        288856.0      4
M-LC       XY24      205688.0      4
H-LC       AR        194809.0      4
M-SC       X40       194086.0      2
X-LC       AR        183427.0      3
H-MC       XY24      170396.0      2
L-SC       XR        161581.0      2
X-MC       XY24      145206.0      2
H-MC       AR        133563.0      2
X-SC       SR        120422.0      1
           XR        117725.0      1
           X40N      115740.0      2
M-LC       XR        102813.0      1
X-MC       X40N       98925.0      2
M-SC       AR         93127.0      1
H-SC       X40        87471.0      2
M-MC       XR         85873.0      1
L-SC       XY24       84131.0      1
M-SC       SR         83776.0      1
M-MC       AR         81782.0      1
X-LC       XY25       72062.0      3
H-MC       X40        69744.0      2
X-LC       X200       66928.0      1
L-SC       AR         64130.0      1
H-MC       XR         58998.0      1
X-MC       X40        55937.0      1
H-MC       X40N       40360.0      1
H-LC       XY25       28455.0      1
M-LC       XY25       18728.0      1
X-MC       AR          9292.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
